# 06 Explore Principals Data


## 06.01 Imports


### 06.01.01 Python Imports


In [110]:
import pandas as pd
import numpy as np
import gzip

### 06.01.02 Import Principal History Data


In [111]:
History = pd.read_csv('../Bens_Data/imdb_scrape_director_writer_hist.csv')
History.drop(columns=['Unnamed: 0'], inplace=True)

In [112]:
History.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7837 entries, 0 to 7836
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tconst      7837 non-null   object
 1   newurl      7837 non-null   object
 2   mpaarating  7837 non-null   object
 3   rlsdt       7837 non-null   object
 4   budget      7837 non-null   object
 5   wordlwide   7837 non-null   object
dtypes: object(6)
memory usage: 367.5+ KB


In [113]:
History.head()

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
0,tt0113403,https://www.imdb.com/title/tt0113403,R,"February 16, 1996 (United States)",error,"$469,571"
1,tt0450972,https://www.imdb.com/title/tt0450972,PG,"September 21, 2007 (United Kingdom)",error,"$563,162"
2,tt0475331,https://www.imdb.com/title/tt0475331,error,"December 13, 2006 (France)","$27,000,000 (estimated)","$2,000,853"
3,tt0800369,https://www.imdb.com/title/tt0800369,PG-13,"May 6, 2011 (United States)","$150,000,000 (estimated)","$449,326,618"
4,tt11229040,https://www.imdb.com/title/tt11229040,error,error,error,error


## 06.02 Clean Up Principal History


### 06.02.01 Remove TV Shows


Since we are only concerned with movies, we want to drop anything that is TV or TV Related.  With more time, we would try to incporate some stats around TV history and then do some PCA to determine it would be worth including in our model.  Presumably, some history with TV might mean more success in film.

In [97]:
History['mpaarating'].value_counts()

error        3234
Approved      888
PG            850
PG-13         674
R             671
Passed        551
Not Rated     412
G             380
Unrated        56
GP             20
X              18
M               4
M/PG            4
NC-17           2
Open            1
Name: mpaarating, dtype: int64

In [98]:
History = History.drop(History[(History['mpaarating'].str.contains("TV"))].index)
History['mpaarating'].value_counts()

error        3234
Approved      888
PG            850
PG-13         674
R             671
Passed        551
Not Rated     412
G             380
Unrated        56
GP             20
X              18
M               4
M/PG            4
NC-17           2
Open            1
Name: mpaarating, dtype: int64

In [99]:
History.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7765 entries, 0 to 7836
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tconst      7765 non-null   object
 1   newurl      7765 non-null   object
 2   mpaarating  7765 non-null   object
 3   rlsdt       7765 non-null   object
 4   budget      7765 non-null   object
 5   wordlwide   7765 non-null   object
dtypes: object(6)
memory usage: 682.7+ KB


### 06.02.02 Exploring MPAA Ratings


### 'Approved' and 'Passed' pre-date the current MPAA rating system.  
 - 'Passed' what was known as the 'Haynes Code' (1930-1934) 
 - 'Open' was also used in 1934
 - 'Approved' for exhibition (1934-1968)   
 - G, M, R, and X were used from 1968 to 1970 'M' meant "Suggested for mature audiences - Parental discretion advised"
 - G, GP, R, X were used from 1970 to 1972 'GP' meant "All ages admitted – Parental guidance suggested."
 - G, PG, R, X were used from 1973 to 1984 PG meant "Parental guidance suggested – Some material may not be suitable for pre-teenagers."
 - PG-13 was added in 1984 PG-13 meant "Parents strongly cautioned – Some material may be inappropriate for children under 13"
 - NC-17 Replaced X in 1990
 - M, M/PG were used by some films originating outside the U.S.
 
### We can probably smooth these out like this, or just create dummy columns.
  - 1 : Passed, Approved, Open, G
  - 2 : GP, PG
  - 2.5 : PG-13
  - 3 : R
  - 4 : NC-17, X
  - 100 : Not Rated, Unrated, and error (error meant the field was missing entirely from IMDB)
  - REMOVE : M, M/PG 
 
 With help from https://en.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system#Replacement_of_the_Hays_Code

In [66]:
History[History['mpaarating'] == 'Not Rated']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
53,tt2531318,https://www.imdb.com/title/tt2531318,Not Rated,"September 15, 2016 (United States)",error,"$12,283,966"
62,tt0074486,https://www.imdb.com/title/tt0074486,Not Rated,"February 3, 1978 (United States)","$10,000 (estimated)","$23,557"
138,tt0057047,https://www.imdb.com/title/tt0057047,Not Rated,"March 28, 1963 (United States)",error,error
142,tt0059170,https://www.imdb.com/title/tt0059170,Not Rated,"August 6, 1965 (United States)","$45,000 (estimated)",error
258,tt0082043,https://www.imdb.com/title/tt0082043,Not Rated,"November 12, 1982 (New Zealand)",error,error
...,...,...,...,...,...,...
7642,tt3483646,https://www.imdb.com/title/tt3483646,Not Rated,"March 14, 2014 (India)",error,"$112,986"
7643,tt3495026,https://www.imdb.com/title/tt3495026,Not Rated,"April 15, 2016 (United States)","$15,670,000 (estimated)","$28,240,000"
7691,tt2807410,https://www.imdb.com/title/tt2807410,Not Rated,"October 18, 2015 (India)",error,"$11,229,399"
7738,tt1442570,https://www.imdb.com/title/tt1442570,Not Rated,"September 25, 2009 (India)",error,error


In [67]:
History[History['mpaarating'] == 'Open']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
3415,tt0024968,https://www.imdb.com/title/tt0024968,Open,"September 12, 1934 (United States)","$100,000 (estimated)",error


In [68]:
History[History['mpaarating'] == 'M']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
2520,tt0064217,https://www.imdb.com/title/tt0064217,M,"May 8, 1969 (United States)",error,error
3964,tt0066534,https://www.imdb.com/title/tt0066534,M,"February 28, 1969 (Italy)",error,"$2,893"
5942,tt0064045,https://www.imdb.com/title/tt0064045,M,"March 21, 1969 (United Kingdom)",error,error
7013,tt0064415,https://www.imdb.com/title/tt0064415,M,"February 8, 1970 (United Kingdom)",error,error


In [69]:
History[History['mpaarating'] == 'M/PG']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
3001,tt0064808,https://www.imdb.com/title/tt0064808,M/PG,1969 (United States),error,error
3486,tt0059535,https://www.imdb.com/title/tt0059535,M/PG,"July 29, 1968 (United States)",error,error
6460,tt0064840,https://www.imdb.com/title/tt0064840,M/PG,"February 25, 1969 (United Kingdom)",error,$30
7239,tt0065025,https://www.imdb.com/title/tt0065025,M/PG,"May 28, 1969 (United States)",error,error


### 06.02.03 Exploring Release Dates


We do have release information for most films, not all.  

In [70]:
History[History['rlsdt'] == 'error']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
4,tt11229040,https://www.imdb.com/title/tt11229040,error,error,error,error
19,tt1582272,https://www.imdb.com/title/tt1582272,error,error,error,error
22,tt2475426,https://www.imdb.com/title/tt2475426,error,error,error,error
44,tt10551162,https://www.imdb.com/title/tt10551162,error,error,error,error
45,tt11708502,https://www.imdb.com/title/tt11708502,error,error,error,error
...,...,...,...,...,...,...
7832,tt13650686,https://www.imdb.com/title/tt13650686,error,error,error,error
7833,tt4357192,https://www.imdb.com/title/tt4357192,error,error,error,error
7834,tt4357320,https://www.imdb.com/title/tt4357320,error,error,error,error
7835,tt4955772,https://www.imdb.com/title/tt4955772,error,error,error,error


### 06.02.04 Exploring Budget Data


We are missing a lot of budget information.  

In [71]:
History[History['budget'] == 'error']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
0,tt0113403,https://www.imdb.com/title/tt0113403,R,"February 16, 1996 (United States)",error,"$469,571"
1,tt0450972,https://www.imdb.com/title/tt0450972,PG,"September 21, 2007 (United Kingdom)",error,"$563,162"
4,tt11229040,https://www.imdb.com/title/tt11229040,error,error,error,error
5,tt12789558,https://www.imdb.com/title/tt12789558,PG-13,"November 12, 2021 (United States)",error,"$46,922,870"
7,tt3089630,https://www.imdb.com/title/tt3089630,PG,"June 12, 2020 (United States)",error,error
...,...,...,...,...,...,...
7832,tt13650686,https://www.imdb.com/title/tt13650686,error,error,error,error
7833,tt4357192,https://www.imdb.com/title/tt4357192,error,error,error,error
7834,tt4357320,https://www.imdb.com/title/tt4357320,error,error,error,error
7835,tt4955772,https://www.imdb.com/title/tt4955772,error,error,error,error


### 06.02.05 Exploring Revenue Data


Of our 7765 rows, 5569 have no worldwide revenue numbers.

In [72]:
History[History['wordlwide'] == 'error']

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide
4,tt11229040,https://www.imdb.com/title/tt11229040,error,error,error,error
7,tt3089630,https://www.imdb.com/title/tt3089630,PG,"June 12, 2020 (United States)",error,error
10,tt0082910,https://www.imdb.com/title/tt0082910,R,"November 5, 1982 (United States)","$145,786 (estimated)",error
19,tt1582272,https://www.imdb.com/title/tt1582272,error,error,error,error
20,tt1630029,https://www.imdb.com/title/tt1630029,error,"December 16, 2022 (United States)",error,error
...,...,...,...,...,...,...
7832,tt13650686,https://www.imdb.com/title/tt13650686,error,error,error,error
7833,tt4357192,https://www.imdb.com/title/tt4357192,error,error,error,error
7834,tt4357320,https://www.imdb.com/title/tt4357320,error,error,error,error
7835,tt4955772,https://www.imdb.com/title/tt4955772,error,error,error,error


## 06.03 IMDB Rating of Principal History


Given how much information is missing from our writer and director history, instead of trying to show a track record with ROI, maybe we can use just the revenue numbers and the IMDB rating of their previous work.

### 06.03.01 IMDB Rating Imports


In [73]:
rt=gzip.open('../Other Source Data/IMDB/title.ratings.tsv.gz','rb')
df_ratings = pd.read_csv(rt,sep='\t', low_memory=False)
df_ratings.head()
# # Index	tconst	averageRating	numVotes
# # 0	tt0000001	5.7	1868
# # 1	tt0000002	5.9	247
# # 2	tt0000003	6.5	1640
# # 3	tt0000004	5.8	159
# # 4	tt0000005	6.2	2463

,tconst,averageRating,numVotes
0,tt0000001,5.7,1868
1,tt0000002,5.9,247
2,tt0000003,6.5,1640
3,tt0000004,5.8,159
4,tt0000005,6.2,2463


### 06.03.02 Combining History & Rating Imports


In [74]:
History_Updated = pd.merge(History ,                 # left df
                          df_ratings,                  # right df
                          how="left",                 # left join
                          left_on='tconst',            # left column
                          right_on='tconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

In [75]:
History_Updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7765 entries, 0 to 7764
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   tconst         7765 non-null   object  
 1   newurl         7765 non-null   object  
 2   mpaarating     7765 non-null   object  
 3   rlsdt          7765 non-null   object  
 4   budget         7765 non-null   object  
 5   wordlwide      7765 non-null   object  
 6   averageRating  6003 non-null   float64 
 7   numVotes       6003 non-null   float64 
 8   _merge         7765 non-null   category
dtypes: category(1), float64(2), object(6)
memory usage: 553.7+ KB


In [76]:
History_Updated['_merge'].value_counts(normalize=True)

both          0.773084
left_only     0.226916
right_only    0.000000
Name: _merge, dtype: float64

### 06.03.03 Exploring Errors


In [77]:
History_Updated[(History_Updated['averageRating'].isnull() == 1) & (History_Updated['rlsdt'] != "error")].head(25)

,tconst,newurl,mpaarating,rlsdt,budget,wordlwide,averageRating,numVotes,_merge
20,tt1630029,https://www.imdb.com/title/tt1630029,error,"December 16, 2022 (United States)",error,error,NaN,NaN,left_only
21,tt1757678,https://www.imdb.com/title/tt1757678,error,"December 20, 2024 (United States)",error,error,NaN,NaN,left_only
23,tt3095356,https://www.imdb.com/title/tt3095356,error,"December 18, 2026 (United States)",error,error,NaN,NaN,left_only
24,tt5637536,https://www.imdb.com/title/tt5637536,error,"December 22, 2028 (United States)",error,error,NaN,NaN,left_only
46,tt12156974,https://www.imdb.com/title/tt12156974,error,"March 19, 2022 (United States)",error,error,NaN,NaN,left_only
47,tt12262116,https://www.imdb.com/title/tt12262116,PG-13,"November 18, 2022 (United States)",error,error,NaN,NaN,left_only
102,tt14208870,https://www.imdb.com/title/tt14208870,error,"November 23, 2022 (United States)",error,error,NaN,NaN,left_only
132,tt2049403,https://www.imdb.com/title/tt2049403,error,June 2025 (United States),error,error,NaN,NaN,left_only
177,tt9419884,https://www.imdb.com/title/tt9419884,PG-13,"May 6, 2022 (United States)",error,error,NaN,NaN,left_only
200,tt4593060,https://www.imdb.com/title/tt4593060,error,September 2022 (United States),"$150,000,000 (estimated)",error,NaN,NaN,left_only


Given that most of these are from the 1920's (long before IMDB) or they are in pre-development (not yet released), I'm ok still using this data set.  We can do some analysis on the which data is missing and maybe devise a method to collect that data in the future.

### 06.03.03 Removing TV data from Combined History


We previsouly removed 72 rows above for being TV projects. We'll need to remove those from our combined history table as well.

In [100]:
directors_writers_combined_history = pd.read_csv('../Bens_Data/directors_writers_combined_history.csv')
directors_writers_combined_history.drop(columns='Unnamed: 0',inplace=True)

In [101]:
directors_writers_combined_history.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres
0,tt0113403,movie,A Midwinter's Tale,1995,99,Comedy
1,tt0450972,movie,As You Like It,2006,127,"Comedy,Drama,Romance"
2,tt0475331,movie,The Magic Flute,2006,135,"Drama,Musical,Romance"
3,tt0800369,movie,Thor,2011,115,"Action,Adventure,Fantasy"
4,tt11229040,movie,Untitled Bee Gees Biopic,2022,\N,"Biography,Drama,Music"


## 06.04 Merge History List with Stats from Each Film


In [102]:
directors_writers_combined_history = pd.merge(directors_writers_combined_history ,                 # left df
                          History_Updated[['tconst', 'newurl', 'mpaarating', 'rlsdt', 'budget', 'wordlwide', 'averageRating', 'numVotes']],                  # right df
                          how="left",                 # left join
                          left_on='tconst',            # left column
                          right_on='tconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        ) 

### 06.04.01 Double Checknig Merge


In [103]:
directors_writers_combined_history.head()

,tconst,titleType,primaryTitle,startYear,runtimeMinutes,genres,newurl,mpaarating,rlsdt,budget,wordlwide,averageRating,numVotes,_merge
0,tt0113403,movie,A Midwinter's Tale,1995,99,Comedy,https://www.imdb.com/title/tt0113403,R,"February 16, 1996 (United States)",error,"$469,571",7.2,2577.0,both
1,tt0450972,movie,As You Like It,2006,127,"Comedy,Drama,Romance",https://www.imdb.com/title/tt0450972,PG,"September 21, 2007 (United Kingdom)",error,"$563,162",6.1,3354.0,both
2,tt0475331,movie,The Magic Flute,2006,135,"Drama,Musical,Romance",https://www.imdb.com/title/tt0475331,error,"December 13, 2006 (France)","$27,000,000 (estimated)","$2,000,853",6.5,1236.0,both
3,tt0800369,movie,Thor,2011,115,"Action,Adventure,Fantasy",https://www.imdb.com/title/tt0800369,PG-13,"May 6, 2011 (United States)","$150,000,000 (estimated)","$449,326,618",7.0,810857.0,both
4,tt11229040,movie,Untitled Bee Gees Biopic,2022,\N,"Biography,Drama,Music",https://www.imdb.com/title/tt11229040,error,error,error,error,NaN,NaN,both


In [104]:
directors_writers_combined_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7837 entries, 0 to 7836
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   tconst          7837 non-null   object  
 1   titleType       7837 non-null   object  
 2   primaryTitle    7837 non-null   object  
 3   startYear       7837 non-null   object  
 4   runtimeMinutes  7837 non-null   object  
 5   genres          7837 non-null   object  
 6   newurl          7765 non-null   object  
 7   mpaarating      7765 non-null   object  
 8   rlsdt           7765 non-null   object  
 9   budget          7765 non-null   object  
 10  wordlwide       7765 non-null   object  
 11  averageRating   6003 non-null   float64 
 12  numVotes        6003 non-null   float64 
 13  _merge          7837 non-null   category
dtypes: category(1), float64(2), object(11)
memory usage: 865.0+ KB


In [105]:
directors_writers_combined_history._merge.value_counts(normalize=True)

both          0.990813
left_only     0.009187
right_only    0.000000
Name: _merge, dtype: float64

In [106]:
directors_writers_combined_history = directors_writers_combined_history.drop(directors_writers_combined_history[(directors_writers_combined_history['_merge'].str.contains("left_only"))].index)

In [107]:
directors_writers_combined_history._merge.value_counts(normalize=True)

both          1.0
left_only     0.0
right_only    0.0
Name: _merge, dtype: float64

In [108]:
directors_writers_combined_history._merge.value_counts()

both          7765
left_only        0
right_only       0
Name: _merge, dtype: int64

## 06.05 Export Directors/Writers Combined History Data


In [109]:
directors_writers_combined_history.to_csv('../Bens_Data/directors_writers_combined_history_updated.csv')